In [12]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import inspect
import scipy
from scipy.stats import entropy

In [2]:
config = {"input_size" : 784,
          "hidden_layer1" : 512,
          "hidden_layer2" : 256,
          "bottleneck" : 128,
          "hidden_layer3" : 256,
          "hidden_layer4" : 512,
          "learning_rate" : 0.001,
          "number_epoch" : 100,
          "batch_size" : 100
          }

In [3]:
transformation = transforms.Compose([transforms.ToTensor()])

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transformation, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transformation, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=config["batch_size"], shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=config["batch_size"], shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:01<00:00, 8.48MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 340kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.18MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.03MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [109]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, input_size=config["input_size"], hidden_layer1=config["hidden_layer1"], hidden_layer2=config["hidden_layer2"], bottleneck=config["bottleneck"], hidden_layer3=config["hidden_layer3"], hidden_layer4=config["hidden_layer4"]):
        super(VariationalAutoencoder, self).__init__()

        self.encoding_layers = nn.Sequential(
            nn.Linear(input_size, hidden_layer1),
            nn.ReLU(),
            nn.Linear(hidden_layer1, hidden_layer2),
            nn.ReLU(),
            nn.Linear(hidden_layer2, bottleneck)
        )

        self.decoding_layers = nn.Sequential(
            nn.Linear(bottleneck, hidden_layer3),
            nn.ReLU(),
            nn.Linear(hidden_layer3, hidden_layer4),
            nn.ReLU(),
            nn.Linear(hidden_layer4, input_size)
        )  
    def encoder_forward(self, data_vector):
        # make some error handling here later hopefully 
        # q_phi(z|x) is what we want to calculate here, we assume q_phi(z|x) is a from a normal
        # we must find mew and sigma^2 for the normal 
        # to find the following we take the data vector and put in its latent representation  
        # q_phi(z|x) is approximatly N_phi(z|x) with paramters mew and sigma as phi

        encoder_output = self.encoding_layers(data_vector).detach().numpy()
    

        mew = np.sum(encoder_output, axis=0)
        log_sigma = np.log(np.sqrt(np.var(encoder_output, axis=0)))

        return mew,  log_sigma
    def decoder_forward(self, mew, log_sigma):
        #z = u + sigma * eita^i
        #eita ~ N(0, 1)

        eita = scipy.stats.norm.rvs(loc=0, scale=1, size=config['bottleneck'])
        latent_space =  scipy.stats.norm.rvs(loc=mew, scale=abs(log_sigma*eita), size=config['batch_size'])
        print(latent_space.shape)


        for i in range(config['bottleneck']):
            log_sigma_eita = abs(log_sigma[i] * eita[i])
            latent_space[i] = scipy.stats.norm.rvs(loc=mew[i], scale=(log_sigma_eita))

        latent_space = torch.from_numpy(latent_space).reshape(-1, config["bottleneck"]).type(torch.float32)
        
        decoder_output = self.decoding_layers(latent_space)

        return decoder_output


In [110]:
def Cost(data_vector, output_vector, mew, log_sigma):
    L_2 = nn.MSELoss(data_vector, output_vector)
    K_L = np.log(1/log_sigma) + ((log_sigma**2 + mew**2)/2) - 0.5
    return L_2 + K_L

In [116]:
Model = VariationalAutoencoder()
sig = inspect.signature(VariationalAutoencoder.__init__)
keys_list = [key for key in sig.parameters.keys() if key != 'self']
Model = VariationalAutoencoder(*[config[key] for key in keys_list])
optimizer = optim.Adam(Model.parameters(), lr=config["learning_rate"])
losses = []
for epoch in range(config["number_epoch"]):
    epoch_loss = []
    for (i, (image_matrix, label) )in enumerate(train_loader):       
        #Forward pass
        image_vector = image_matrix.reshape(-1, config["input_size"])

        image_vector = image_vector.type(torch.float32)


        mew, log_sigma = Model.encoder_forward(image_vector)
        print(log_sigma.shape)


        reconstructed_image = Model.decoder_forward(mew, log_sigma)


        loss = Cost(image_vector, reconstructed_image, mew, log_sigma)
        losses.append(loss.item())
        epoch_loss.append(loss.item())
        #backpropogation

        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 

        avg_loss = sum(epoch_loss) / len(epoch_loss)
        if i % 100 == 0:
            print(f'Epoch [{epoch+1}/{config["number_epoch"]}], Avg Loss: {avg_loss:.6f}')


(128,)


ValueError: size does not match the broadcast shape of the parameters. 100, (100,), (128,)